In [2]:
import numpy as np # linear algebra
import seaborn as sns
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import tensorflow as tf
import os

In [9]:
path = os.getcwd() + '\kc_house_data.csv'
h_data = pd.read_csv(path)

In [10]:
h_data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


array([[7129300520, '20141013T000000', 221900.0, ..., -122.257, 1340,
        5650],
       [6414100192, '20141209T000000', 538000.0, ..., -122.319, 1690,
        7639],
       [5631500400, '20150225T000000', 180000.0, ...,
        -122.23299999999999, 2720, 8062],
       ...,
       [1523300141, '20140623T000000', 402101.0, ...,
        -122.29899999999999, 1020, 2007],
       [291310100, '20150116T000000', 400000.0, ..., -122.069, 1410,
        1287],
       [1523300157, '20141015T000000', 325000.0, ...,
        -122.29899999999999, 1020, 1357]], dtype=object)

In [13]:
h_data.pop('floors');h_data.pop('waterfront');h_data.pop('view');h_data.pop('condition');
h_data.pop('grade');h_data.pop('sqft_above');h_data.pop('sqft_basement');h_data.pop('yr_built');h_data.pop('yr_renovated');h_data.pop('zipcode');
h_data.pop('lat');h_data.pop('sqft_living15');h_data.pop('sqft_lot15');h_data.pop('id');h_data.pop('date');


In [19]:
h_data.head()
h_data.describe()

,price,bedrooms,bathrooms,sqft_living,sqft_lot
count,2.161300e+04,21613.000000,21613.000000,21613.000000,2.161300e+04
mean,5.400881e+05,3.370842,2.114757,2079.899736,1.510697e+04
std,3.671272e+05,0.930062,0.770163,918.440897,4.142051e+04
min,7.500000e+04,0.000000,0.000000,290.000000,5.200000e+02
25%,3.219500e+05,3.000000,1.750000,1427.000000,5.040000e+03
50%,4.500000e+05,3.000000,2.250000,1910.000000,7.618000e+03
75%,6.450000e+05,4.000000,2.500000,2550.000000,1.068800e+04
max,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06


In [ ]:
x = h_data.drop(labels=['price'], axis=1).values
y = h_data.price.values


In [ ]:
h_data.head()

In [ ]:
x

In [ ]:
y

In [ ]:
seed = 5
np.random.seed(seed)
tf.set_random_seed(seed)

In [ ]:
train_index = np.random.choice(len(x), round(len(x) * 0.5), replace=False)

In [ ]:
test_index = np.array(list(set(range(len(x))) - set(train_index)))
train_X = x[train_index]
train_y = y[train_index]
test_X = x[test_index]
test_y = y[test_index]

In [ ]:
train_X[:5]

In [ ]:
def min_max_normalized(data):
    col_max = np.max(data, axis=0)
    col_min = np.min(data, axis=0)
    return np.divide(data - col_min, col_max - col_min)

In [ ]:
train_X = min_max_normalized(train_X)
test_x = min_max_normalized(test_X)

In [ ]:
data = tf.placeholder(dtype=tf.float32, shape=[None, 4], name='Data_Input')
labels = tf.placeholder(dtype=tf.float32, shape=[None, 1], name='Labels_Input')

In [ ]:
data

In [ ]:
data.shape

In [ ]:
labels_size = 1
logits = tf.layers.dense(inputs=data, units=1)

In [ ]:
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=labels))
tf.summary.scalar('loss', loss)

In [ ]:
learning_rate = 0.05
batch_size = 20
iter_num = 10000

In [ ]:
opt = tf.train.GradientDescentOptimizer(learning_rate)

In [ ]:
goal = opt.minimize(loss)

In [ ]:
# Define the accuracy
# The default threshold is 0.5, rounded off directly
prediction = tf.round(tf.sigmoid(logits), name='Prediction')
# Bool into float32 type
correct = tf.cast(tf.equal(prediction, labels), dtype=tf.float32)
# Average
accuracy = tf.reduce_mean(correct, name='Accuracy')

tf.summary.scalar('accuracy', accuracy)
# End of the definition of the model framework

In [ ]:
merged = tf.summary.merge_all()

In [ ]:
loss_trace = []
train_acc = []
test_acc = []

In [ ]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [ ]:
train_writer = tf.summary.FileWriter('./house_logistic_regression', sess.graph)

In [ ]:
# training model
for epoch in range(iter_num):
    
    # Generate random batch index
    batch_index = np.random.choice(len(train_X), size=batch_size)
    batch_train_X = train_X[batch_index]
    batch_train_y = np.matrix(train_y[batch_index]).T
    
    
    _, summary = sess.run([goal, merged], feed_dict={data: batch_train_X, labels: batch_train_y})
    
    
    temp_loss = sess.run(loss, feed_dict={data: batch_train_X, labels: batch_train_y})
    # convert into a matrix, and the shape of the placeholder to correspond
    
    train_logit, train_pred, train_corr = sess.run([logits, prediction, correct],
                                                   feed_dict={data: test_X, labels: np.matrix(test_y).T})

    temp_train_acc = sess.run(accuracy, feed_dict={data: train_X, labels: np.matrix(train_y).T})
    temp_test_acc = sess.run(accuracy, feed_dict={data: test_X, labels: np.matrix(test_y).T})
    # recode the result
    loss_trace.append(temp_loss)
    train_acc.append(temp_train_acc)
    test_acc.append(temp_test_acc)
    train_writer.add_summary(summary, epoch)
    # output
    if (epoch + 1) % 300 == 0:
        print('epoch: {:4d} loss: {:5f} train_acc: {:5f} test_acc: {:5f}'.format(epoch + 1, temp_loss,
                                                                          temp_train_acc, temp_test_acc))